In [1]:
# Basic constants

OUTPUT_DIR = '/home/ubuntu/visual_search_assistant/M3/results'
FACE_LIB_DIR = '/home/ubuntu/visual_search_assistant/M3/library/'
DATA_DIR = '/home/ubuntu/visual_search_assistant/data'
SAMPLE_FRAME_FREQ = 2
SAMPLE_CLUSTER_FREQ = 20
LOG_FREQ = 50

batch_size = 128

In [2]:
!export OPENBLAS_CORETYPE=ARMV8

## Library of face embeddings

In [ ]:
import os
# import face_recognition
import pickle

meta_pth = '/dli/task/visual_search_assistant/M3/library/meta.npz' #'/home/ubuntu/visual_search_assistant/M3/library/meta.npz'
all_embeddings = []
all_names = []
with open(meta_pth,'rb') as f:
    metadata = pickle.load(f)
    
    for idx,(key,val) in enumerate(metadata.items()):
        all_embeddings.append(val['encoding'])
        all_names.append(key)

print("✔️ List of all_names: \n",all_names)
print("✔️ Shape of single face encoding: \n", all_embeddings[0].shape)

In [ ]:
import os
import cv2
import pickle
import numpy as np


def process_video(input_pth,output_pth=None,use_gpu=True,recognition=False,detection_threshold=0.7,
                  result_dir=None,max_frames=None,sample_cluster_freq=2,sample_freq=2,batch_size=1):
    if output_pth is None:
        output_pth = os.path.join(OUTPUT_DIR,input_pth.split('/')[-1])
    if use_gpu:
        batch_size = 16
    
    if result_dir is None:
        import pdb;pdb.set_trace()
        input_name = input_pth.split('/')[-1].split('.')[0]
        
        result_dir = os.path.join(OUTPUT_DIR,input_name)
    
    if not os.path.exists(result_dir):
        sampled_face_dir = os.path.join(result_dir,'sampled_faces')
        os.makedirs(sampled_face_dir)
    else:
        print('Error result dir %s already exists! aborting' % result_dir)
        sampled_face_dir = os.path.join(result_dir,'sampled_faces')
#         return
    
    
    cluster_dict = {}
    
    video_capture = cv2.VideoCapture(input_pth)

    frame_width = int(video_capture.get(3))
    frame_height = int(video_capture.get(4))
    
    fps = video_capture.get(cv2. CAP_PROP_FPS)
    nframes = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    length = nframes/fps
    
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter(output_pth, fourcc, 30.0, (frame_width, frame_height))
    
    frame_count = 0
    
    frames = []
    print('='*20,'Start Face Detection and Recognition','='*20)
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        
        if not ret or (max_frames and frame_count > max_frames):
            break
        if frame_count % LOG_FREQ == LOG_FREQ -1:
            print('Processed %d frames'% frame_count)
            
        frame_count += 1
        
        # skip frames
        if frame_count % sample_freq > 0:
            continue
            
        frames.append(frame)
        if len(frames) == batch_size:
#             import pdb;pdb.set_trace()
            batch_face_locations = face_recognition.batch_face_locations(frames)
            for frame_idx,face_locations in enumerate(batch_face_locations):
                print('reaches here')
                number_of_faces_in_frame = len(face_locations)
                
                fno = frame_count - batch_size + frame_idx
                frame = frames[frame_idx]
                
                
                embeddings = face_recognition.face_encodings(frame,face_locations)
                for face_idx,(embd,(top,right,bottom,left)) in enumerate(zip(embeddings,face_locations)):
                    # Draw a box around the face
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    # Draw a label with a name below the face
                    cv2.rectangle(frame, (left, bottom + 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    
                    
                    # write label
                    face_dist = face_recognition.face_distance(all_embeddings,embd)
                    label_idx = np.argmin(face_dist)
                    name = all_names[label_idx] if face_dist[label_idx] > detection_threshold else 'Unknown'
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom + 6), font, 0.5, (255, 255, 255), 1)
                    
                    
                    # save face image for clustering
                    if frame_idx % sample_cluster_freq ==0:
                        fname = '%.3d_%.3d.png'%(fno,face_idx)
                        face_img = frame[top:bottom,left:right,:]
                        # save face image
                        cv2.imwrite(os.path.join(sampled_face_dir,fname),face_img)
                        # save embd in dict
                        cluster_dict[fname.split('.')[0]] = embd
                                
                out.write(frame)
            frames = []
                
                
    video_capture.release()
    out.release()
    
    # save sampled embeddings as pickle file
#     import pdb;pdb.set_trace()
    cluster_meta_file = os.path.join(result_dir,'embeddings.pickle')
    with open(cluster_meta_file,'wb') as f:
        pickle.dump(cluster_dict,f,protocol=pickle.HIGHEST_PROTOCOL)
    
    print('='*15,'Done Processing %s to %s' % (input_pth,output_pth),'='*15)
    
    return name,nframes,length

In [ ]:
test_input_video = '/dli/task/visual_search_assistant/M3/data/parks_and_rec_0_10.mp4' #'/home/ubuntu/visual_search_assistant/data/radio_star_10_20.mp4'
vid_name,vid_nframes,vid_length = process_video(test_input_video,max_frames=64,batch_size=16)

### Check sampled faces & embeddings

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import glob

def display_multiple_images(images,names,columns=10,figsize=(20,10)):
    plt.figure(figsize=figsize)
    plt.subplots_adjust(wspace=0, hspace=0)
    for i, (image,name) in enumerate(zip(images,names)):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.imshow(image)
        plt.axis('off')
        plt.title(name) 
        

num_images = 30
%matplotlib inline

print('='*10,'Display sampled faces','='*10)
print("title is in <(frame number)_(detected face id)> format")

images = []
names = []
for img_path in glob.glob(os.path.join(result_dir,'sampled_faces','*.png')):
    images.append(mpimg.imread(img_path))
    names.append(img_path.split('/')[-1].split('.')[0])
    if len(images) == num_images:
        break
        
        
display_multiple_images(images,names)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
embd_pkl_file = '/home/ubuntu/visual_search_assistant/M3/results/radio_star_10_20/embeddings.pickle'
result_dir = '/home/ubuntu/visual_search_assistant/M3/results/radio_star_10_20'

import pickle
import numpy as np

with open(embd_pkl_file,'rb') as rf:
    embedding_dict = pickle.load(rf)

embedding_dict.keys()

## Cluster Some Faces!
Use a clustering algorithm from Scikit
https://pyimagesearch.com/2018/07/09/face-clustering-with-python/

In [2]:
embeddings = np.array(list(embedding_dict.values()))
names = np.array(list(embedding_dict.keys()))

NameError: name 'np' is not defined

In [1]:
from sklearn.cluster import DBSCAN

clt = DBSCAN(metric="euclidean", n_jobs=-1)
clt.fit(embeddings)

# determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

print("Save clustered results in dict")
cluster = {}
# save separately as clusters
for cluster_idx in labelIDs:
    idxs = np.where(clt.labels_ == cluster_idx)
    cluster[cluster_idx] = names[idxs]

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import cv2

# helper function for converting fnames to images for using imutils build_montages function
def fnames2images(fnames,dir=None):
    faces = []
    for fname in fnames:
        if not fname.endswith('.png'):
            fname = fname + '.png'
        image = cv2.imread(os.path.join(dir,fname))
        face = cv2.resize(image, (96,96))
        faces.append(face)
    return faces

#### Visualize the cluster results!

In [ ]:
from imutils import build_montages

plt.figure(figsize=(10,5))
plt.subplots_adjust(wspace=0, hspace=0)

for idx,(key,name_list) in enumerate(cluster.items()):
    faces = fnames2images(name_list,dir=os.path.join(result_dir,'sampled_faces'))
    montage = build_montages(faces,(96,96),(5,5))[0]
    title = "Faces for cluster %d" % key if key!=-1 else "Unknown Faces"
    plt.subplot(1,3,idx+1)
    plt.axis('off')
    plt.title(title)
    plt.imshow(cv2.cvtColor(montage,cv2.COLOR_BGR2RGB))
    

In [ ]:
embeddings.shape

### Write Results to Opensearch

In [ ]:
from search import *

print("="*10, "Check client status")
print(client.info())

INDEX_CLUSTERS = "clusters"
INDEX_VIDEOS = "videos"

# vid_name will be the id
vid_document = {
    'id': vid_name,
    'nframes': vid_nframes,
    'length': vid_length,
}
# cluster name will be id
cluster_document = {
    'id': 'test_0',
    'center': embeddings[0],
    'num_faces': 40,
    'list_frames':['000_001','000_003']
}

print("Send results for the new video")
create_or_update(INDEX_VIDEOS,vid_name,vid_document)

print("Send results for each cluster")
create_or_update(INDEX_CLUSTERS,'test_0',cluster_document)


Deleting an index (shouldn't need this...)

In [ ]:
# client.indices.delete(index="videos")